In [17]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [18]:
import os
import torch
import numpy as np
import random
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import torchvision.transforms as T
import torchvision.utils as utils
from torch import nn
from tqdm import tqdm
from timeit import default_timer as timer
import albumentations as A
from albumentations import ToTensorV2

In [19]:
torch.manual_seed(42)
torch.cuda.manual_seed(42)
random.seed(42)
np.random.seed(42)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [20]:
class DRIVE_dataset(Dataset):
    def __init__(self, img_dir, mask_dir, transform=None):
        self.img_dir = img_dir
        self.mask_dir = mask_dir
        self.transform = transform
        self.images = [img for img in os.listdir(img_dir) if img.endswith('.tif')]
        self.valid_pairs = []
        for img in self.images:
            if 'training' in img or 'test' in img:
                base_name = img.split('_')[0]
                mask_name = f"{base_name}_manual1.gif"
                mask_path = os.path.join(self.mask_dir, mask_name)
                if os.path.exists(mask_path):
                    self.valid_pairs.append((img, mask_name))
        if not self.valid_pairs:
            raise ValueError("No valid image-mask pairs")
        print(f"Found {len(self.valid_pairs)} valid pairs in {img_dir}")

    def __len__(self):
        return len(self.valid_pairs)

    def __getitem__(self, index):
        image_name, mask_name = self.valid_pairs[index]
        image_path = os.path.join(self.img_dir, image_name)
        mask_path = os.path.join(self.mask_dir, mask_name)

        image = np.array(Image.open(image_path).convert("RGB"))
        mask = np.array(Image.open(mask_path).convert("L"))

        # Binarize mask: make sure it's 0 and 1
        mask = (mask > 0).astype(np.uint8)

        # If needed, expand mask dims to HWC
        if mask.ndim == 2:
            mask = np.expand_dims(mask, axis=-1)

        if self.transform:
            augmented = self.transform(image=image, mask=mask)
            image = augmented['image']
            mask = augmented['mask']
            mask = mask.float()
            mask = mask.permute(2,0,1)

        return image, mask

In [21]:
class DoubleConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(DoubleConv, self).__init__()
        self.double_conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )
    def forward(self, x):
        return self.double_conv(x)

class Unet(nn.Module):
    def __init__(self, in_channels=3, out_channels=1):
        super(Unet, self).__init__()
        self.enc1 = DoubleConv(in_channels, 64)
        self.enc2 = DoubleConv(64, 128)
        self.enc3 = DoubleConv(128, 256)
        self.enc4 = DoubleConv(256, 512)
        self.bottle_neck = DoubleConv(512, 1024)
        self.pool = nn.MaxPool2d(kernel_size=2)
        self.up1 = nn.ConvTranspose2d(1024, 512, kernel_size=2, stride=2)
        self.dec1 = DoubleConv(1024, 512)
        self.up2 = nn.ConvTranspose2d(512, 256, kernel_size=2, stride=2)
        self.dec2 = DoubleConv(512, 256)
        self.up3 = nn.ConvTranspose2d(256, 128, kernel_size=2, stride=2)
        self.dec3 = DoubleConv(256, 128)
        self.up4 = nn.ConvTranspose2d(128, 64, kernel_size=2, stride=2)
        self.dec4 = DoubleConv(128, 64)
        self.out_conv = nn.Conv2d(64, out_channels, kernel_size=1)

    def forward(self, x):
        x1 = self.enc1(x)
        x2 = self.enc2(self.pool(x1))
        x3 = self.enc3(self.pool(x2))
        x4 = self.enc4(self.pool(x3))
        x = self.bottle_neck(self.pool(x4))
        x = self.up1(x)
        x = torch.cat([x, x4], dim=1)
        x = self.dec1(x)
        x = self.up2(x)
        x = torch.cat([x, x3], dim=1)
        x = self.dec2(x)
        x = self.up3(x)
        x = torch.cat([x, x2], dim=1)
        x = self.dec3(x)
        x = self.up4(x)
        x = torch.cat([x, x1], dim=1)
        x = self.dec4(x)
        return self.out_conv(x)

In [22]:
def iou_score(pred, target, smooth=1e-6):
    pred = torch.sigmoid(pred)
    pred = (pred > 0.5).float()
    target = target.float()

    intersection = (pred * target).sum()
    union = pred.sum() + target.sum() - intersection

    if union == 0:
        return 0.0
    return (intersection + smooth) / (union + smooth)

def dice_score(pred, target, smooth=1e-6):
    pred = torch.sigmoid(pred)
    pred = (pred > 0.5).float()
    target = target.float()
    pred_sum = pred.sum().item()
    target_sum = target.sum().item()
    #print(f"Pred sum: {pred_sum}, Target sum: {target_sum}")
    intersection = (pred * target).sum()
    denominator = pred.sum() + target.sum() + smooth
    if denominator == smooth:
        return 0.0
    return (2. * intersection + smooth) / denominator

def dice_loss(pred, target, smooth=1e-6):
    pred = torch.sigmoid(pred)
    pred = (pred > 0.5).float()
    target = target.float()
    intersection = (pred * target).sum()
    denominator = pred.sum() + target.sum() + smooth
    if denominator == smooth:
        return 1.0
    return 1 - ((2. * intersection + smooth) / denominator)

def combined_loss(y_pred, y_true):
    bce = nn.BCEWithLogitsLoss(pos_weight=torch.tensor([20.0]).to(device))(y_pred, y_true)
    dice = dice_loss(y_pred, y_true)
    return 0.4 * bce + 0.6 * dice

In [23]:

def train_step(model, dataloader, criterion, optimizer):
    model.train()
    train_loss, train_dice,train_iou = 0, 0,0
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        y_pred = model(X)
        loss = criterion(y_pred, y)
        train_loss += loss.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_dice += dice_score(y_pred, y)
        train_iou += iou_score(y_pred,y)
    train_loss /= len(dataloader)
    train_dice /= len(dataloader)
    train_iou /= len(dataloader)
    return train_loss, train_dice,train_iou

In [24]:

def test_step(model, dataloader, criterion, epoch):
    model.eval()
    test_loss, test_dice,test_iou = 0, 0, 0
    with torch.no_grad():
        for batch, (X, y) in enumerate(dataloader):
            X, y = X.to(device), y.to(device)
            y_pred = model(X)
            loss = criterion(y_pred, y)
            test_loss += loss.item()
            test_dice += dice_score(y_pred, y)
            test_iou += iou_score(y_pred,y)
            #if batch == 8 :
                #utils.save_image(pred, f"pred_epoch_{epoch+1}.png")
                #utils.save_image(y, f"target_epoch_{epoch+1}.png")
            if batch == 0:
                pred = torch.sigmoid(y_pred)
                pred = (pred > 0.5).float()

    test_loss /= len(dataloader)
    test_dice /= len(dataloader)
    test_iou /= len(dataloader)
    return test_loss, test_dice,test_iou


In [25]:
def train(model, train_dataloader, test_dataloader, optimizer, criterion, epochs=100):

    results = {"train_loss": [], "train_dice": [],"train_iou": [], "test_loss": [], "test_dice": [],"test_iou":[]}
    #scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5)
    for epoch in tqdm(range(epochs)):
        train_loss, train_dice, train_iou = train_step(model, train_dataloader, criterion, optimizer)
        test_loss, test_dice,test_iou = test_step(model, test_dataloader, criterion, epoch)
        print(f"Epoch: {epoch+1} | train_loss: {train_loss:.4f} | train_dice: {train_dice:.4f} | train_iou : {train_iou:.4f} |"
              f"test_loss: {test_loss:.4f} | test_dice: {test_dice:.4f} | test_iou : {test_iou:.4f}")
        #scheduler.step(test_loss)
        results["train_loss"].append(train_loss)
        results["train_dice"].append(train_dice)
        results["train_iou"].append(train_iou)
        results["test_loss"].append(test_loss)
        results["test_dice"].append(test_dice)
        results["test_iou"].append(test_iou)
    return results


In [26]:
if __name__ == "__main__":
    transform = A.Compose([
    A.Resize(512, 512),
    #A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    #A.RandomBrightnessContrast(p=0.3),
    #A.ShiftScaleRotate(shift_limit=0.02, scale_limit=0.05, rotate_limit=10, p=0.5),
    A.Normalize(mean=(0.5,), std=(0.5,), max_pixel_value=255.0),
    ToTensorV2(),
    ])
    image_transform = T.Compose([
        transform,
        #T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    train_data = DRIVE_dataset(
        img_dir="/content/drive/MyDrive/retina-vessel-segmentation/data/DRIVE/training/images",
        mask_dir="/content/drive/MyDrive/retina-vessel-segmentation/data/DRIVE/training/1st_manual",
        transform=transform
    )
    test_data = DRIVE_dataset(
        img_dir="/content/drive/MyDrive/retina-vessel-segmentation/data/DRIVE/test/images",
        mask_dir="/content/drive/MyDrive/retina-vessel-segmentation/data/DRIVE/test/1st_manual",
        transform=transform
    )
    torch.manual_seed(42)
    torch.cuda.manual_seed(42)
    train_dataloader = DataLoader(train_data, batch_size=2, shuffle=True, num_workers=0)
    test_dataloader = DataLoader(test_data, batch_size=2, shuffle=False, num_workers=0)
    model = Unet(in_channels=3, out_channels=1).to(device)
    criterion = combined_loss
    optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

Found 20 valid pairs in /content/drive/MyDrive/retina-vessel-segmentation/data/DRIVE/training/images
Found 20 valid pairs in /content/drive/MyDrive/retina-vessel-segmentation/data/DRIVE/test/images


In [27]:
start_time = timer()
results = train(model, train_dataloader, test_dataloader, optimizer, criterion, epochs=50)
end_time = timer()
print(f"Total training time: {end_time - start_time:.3f} seconds")

  2%|▏         | 1/50 [00:06<05:36,  6.87s/it]

Epoch: 1 | train_loss: 1.0258 | train_dice: 0.2541 | train_iou : 0.1460 |test_loss: 1.3538 | test_dice: 0.0000 | test_iou : 0.0000


  4%|▍         | 2/50 [00:13<05:33,  6.95s/it]

Epoch: 2 | train_loss: 0.8229 | train_dice: 0.3053 | train_iou : 0.1808 |test_loss: 1.3418 | test_dice: 0.0002 | test_iou : 0.0001


  6%|▌         | 3/50 [00:20<05:25,  6.92s/it]

...

Epoch: 48 | train_loss: 0.2936 | train_dice: 0.7152 | train_iou : 0.5573 |test_loss: 0.4222 | test_dice: 0.7248 | test_iou : 0.5688


 98%|█████████▊| 49/50 [05:51<00:07,  7.17s/it]

Epoch: 49 | train_loss: 0.2737 | train_dice: 0.7424 | train_iou : 0.5912 |test_loss: 0.4128 | test_dice: 0.6966 | test_iou : 0.5348


100%|██████████| 50/50 [05:58<00:00,  7.17s/it]

Epoch: 50 | train_loss: 0.2683 | train_dice: 0.7459 | train_iou : 0.5952 |test_loss: 0.4148 | test_dice: 0.7152 | test_iou : 0.5569
Total training time: 358.384 seconds


In [28]:
torch.save(model.state_dict(), 'unet_weights.pth')
